In [1]:
from langchain_community.document_loaders.csv_loader import CSVLoader
from pathlib import Path
from langchain_openai import ChatOpenAI
from langchain_ollama import OllamaEmbeddings
from dotenv import load_dotenv

In [3]:
load_dotenv()

True

In [4]:
llm = ChatOpenAI(temperature=0, model="meta-llama/llama-3.3-70b-instruct", max_tokens=4000)

In [14]:
file_path = r"E:\AgenticAI Learnings\RAG Techniques\Data\username-or-email.csv"

In [15]:
import pandas as pd

file = pd.read_csv(file_path)
file.head()

,Username,Login_email,Identifier,First name,Last name
0,booker12,rachel@example.com,9012,Rachel,Booker
1,grey07,laura@yourcompany.com,2070,Laura,Grey
2,johnson81,craig@yourcompany.com,4081,Craig,Johnson
3,jenkins46,mary@example.com,9346,Mary,Jenkins
4,smith79,jamie@example.com,5079,Jamie,Smith


In [16]:
loader = CSVLoader(file_path)
docs = loader.load_and_split()

In [17]:
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

embeddings = OllamaEmbeddings(model="nomic-embed-text")
print(len(embeddings.embed_query(" ")))
index = faiss.IndexFlatL2(len(embeddings.embed_query(" ")))

768


In [18]:
vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={}
)

In [19]:
vector_store.add_documents(documents=docs)

['0c25a089-4add-47c7-bec9-3549abc6bcca',
 'd84571a4-1c4f-4d77-8ab6-7c9625ab0fe7',
 'fa0ade6e-1489-4bf1-8f2e-8261ed8abb03',
 '83822615-80a4-42dc-a9e5-a6737f90ec31',
 '8bc0823e-3c06-45e0-815c-e5f6f68cba65']

In [20]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_classic.chains import create_retrieval_chain
from langchain_classic.chains.combine_documents import create_stuff_documents_chain

retriever = vector_store.as_retriever()

# Set up system prompt
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}"),
    
])

# Create the question-answer chain
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [21]:
answer= rag_chain.invoke({"input": "Email id of booker12?"})
answer['answer']

'The email ID of booker12 is rachel@example.com. This belongs to Rachel Booker, with the identifier 9012.'

In [22]:
answer= rag_chain.invoke({"input": "What is the last name of Jamie and give me her identifier?"})
answer['answer']

'The last name of Jamie is Smith, and her identifier is 5079. She can be reached at jamie@example.com. Her username is smith79.'